# Numerical tests for the Landlab GravelBedrockEroder Component

*B.C. Campforts, INSTAAR, University of Colorado Boulder* \
*V. Gabel, CIRES and Department of Geological Sciences, University of Colorado Boulder* \
*G.E. Tucker, CIRES and Department of Geological Sciences, University of Colorado Boulder* 

This notebook describes a series of experiments to test the `GravelBedrockEroder` component. 

In [ ]:
### Imports
import numpy as np
from numpy.testing import assert_almost_equal
from landlab import HexModelGrid
from landlab.components import FlowAccumulator, GravelBedrockEroder
from landlab.plot import imshow_grid
from matplotlib import pyplot as plt

## 1. Plucking

Mathematically, plucking is represented as: 

$$ E_p = a(\tau -\tau_c)^{\frac{3}{2}} e^{\frac{-H}{H_*}}$$
Following near threshold theory this becomes: 
$$ E_p = a(\epsilon\tau_c)^{\frac{3}{2}} e^{\frac{-H}{H_*}}$$

$$ E_p = a\left(\epsilon\theta_c(\sigma -\rho)gD_{50}\right)^{\frac{3}{2}} e^{\frac{-H}{H_*}}$$

From rate into Volume: 
$$ V_p = E_pbΔx $$
where *b* is river width and: 
$$ b = \frac{k_wQS^{\frac{7}{6}}}{D_{50}^{\frac{3}{2}}}$$

$$ V_p = \frac{a(\epsilon\theta_c(\sigma -\rho)gD_{50})^{\frac{3}{2}} e^{\frac{-H}{H_*}}k_wQS^{\frac{7}{6}}Δx}{D_{50}^{\frac{3}{2}}} $$

Where $a(\epsilon\theta_c(\sigma -\rho)g)^{\frac{3}{2}}k_w$ is a constant *c* and average channel incision rate, corrected for numerical cell width equals:  

$$  E̅_p̅ = \frac{V_p}{Δx^2} = \frac{cQS^{\frac{7}{6}}e^{\frac{-H}{H_*}}}{Δx}$$

In [ ]:
from landlab import RasterModelGrid
from landlab.components import FlowAccumulator

nr = 3
nc = 10
grid = RasterModelGrid((nr, nc), xy_spacing=1000.0)
elev = grid.add_zeros("topographic__elevation", at="node")
rock_elev = grid.add_zeros("bedrock__elevation", at="node")
sed = grid.add_zeros("soil__depth", at="node")
sed[grid.core_nodes] = 1
grid.set_closed_boundaries_at_grid_edges(True, True, False, True)

# grid.status_at_node[grid.perimeter_nodes] = grid.BC_NODE_IS_CLOSED
# grid.status_at_node[5] = grid.BC_NODE_IS_FIXED_VALUE
fa = FlowAccumulator(grid, runoff_rate=10.0)
fa.run_one_step()
# TODO Check porosity in deposition (Building walls)
eroder = GravelBedrockEroder(
    grid, abrasion_coefficient=0, plucking_coefficient=1, sediment_porosity=0
)


dt = 1000
U = 1e-3
iterations = int(100)
for _ in range(iterations):
    rock_elev[grid.core_nodes] += U * dt
    elev[grid.core_nodes] += U * dt
    fa.run_one_step()
    eroder.run_one_step(dt)
print("Total uplift is %.02f m" % (iterations * U * dt))
plt.plot(grid.x_of_node[nc : nc + nc - 1], elev[nc : nc + nc - 1], label="Elevation")
plt.plot(grid.x_of_node[nc : nc + nc - 1], rock_elev[nc : nc + nc - 1], label="Bedrock")
plt.plot(grid.x_of_node[nc : nc + nc - 1], sed[nc : nc + nc - 1], label="soil")
plt.legend()

plt.figure()
imshow_grid(grid, "soil__depth", plot_name="soil__depth", color_for_closed="red")
plt.show()
plt.figure()
imshow_grid(grid, "bedrock__elevation", plot_name="bedrock__elevation")
plt.show()
plt.figure()
imshow_grid(grid, "topographic__elevation", plot_name="topographic__elevation")
plt.show()

In [ ]:
grid.x_of_node[nc : nc + nc + 1]

In [ ]:
GravelBedrockEroder